In [2]:
#azims 0, 10, 20, 40, 80
#eleves, 0, 10, 40
#opposite and same sex

array_azims = list(range(-90, 91, 10))
array_elevs = list(range(-20, 41, 10))

In [3]:
azim_deltas = [-80, -40, -20, -10, 0, 10, 20, 40, 80]
elev_deltas = [-40, -10, 0, 10, 40]

azim_elev_combos = dict()

for delta in azim_deltas:
    combos_within = []
    combos_crossed = []
    for azim in array_azims:
        if azim + delta in array_azims:
            azim2 = azim + delta
            if azim * azim2 < 0:
                combos_crossed.append((azim, azim2))
            else:
                combos_within.append((azim, azim2))
    azim_elev_combos[f'azim_{delta}_within'] = combos_within
    azim_elev_combos[f'azim_{delta}_crossed'] = combos_crossed

for delta in elev_deltas:
    combos = []
    for elev in array_elevs:
        if elev + delta in array_elevs:
            elev2 = elev + delta
            combos.append((elev, elev2))
    azim_elev_combos[f'elev_{delta}'] = combos

In [4]:
import pickle

# with open('azim_elev_combos.pkl', 'wb') as f:
#     pickle.dump(azim_elev_combos, f)

In [5]:
azim_elev_combos['azim_-20_crossed']

[(10, -10)]

In [1]:
import random
from copy import deepcopy
import pickle
import pandas

In [2]:
def draw_block(combo_dict):
    """
    drawing set of 90 locations for trials
    """
    trials = []

    cross_dict = {
        80: random.choice([6, 7]),
        40: 2,
        20 : random.choice([0, 1]),
        10: 0,
        0: 0,
        -10: 0,
        -40: 2,
    }
    cross_dict[-80] = 13 - cross_dict[80]
    cross_dict[-20] = 1 - cross_dict[20]

    azim_deltas = [-80, -40, -20, -10, 0, 10, 20, 40, 80]
    elev_deltas = [-40, -10, 0, 10, 40]
    sex = ['same', 'opposite']

    dict_copy = deepcopy(combo_dict)
    for a_delta in azim_deltas:
        for e_delta in elev_deltas:
            for s in sex:
                cross = False
                if cross_dict[a_delta]:
                    cross = True
                    cross_dict[a_delta] -= 1

                if cross:
                    azims = dict_copy[f'azim_{a_delta}_crossed'].pop(random.randrange(len(dict_copy[f'azim_{a_delta}_crossed'])))
                else:
                    azims = dict_copy[f'azim_{a_delta}_within'].pop(random.randrange(len(dict_copy[f'azim_{a_delta}_within'])))

                elevs = random.sample(dict_copy[f'elev_{e_delta}'], 1)[0]
                trials.append(((azims[0], elevs[0]), (azims[1], elevs[1]), s, cross, a_delta, e_delta))
    return trials

In [3]:
def draw_block_0_azim():
    trials = []
    target_locs = [-90, -30, 0, 30, 90]
    distractor_locs = [-90, -30, 0, 30, 90]
    for t in target_locs:
        for d in distractor_locs:
            for s in ['same', 'opposite']:
                if t*d >= 0:
                    cross = False
                else:
                    cross = True
                a_delta = d - t
                trial = ((t, 0), (d, 0), s, cross, a_delta, 0)
                trials.append(trial)
    return trials

In [4]:
def draw_block_elev():
    trials = []

    azims = [-90, -30, 0, 30, 90]
    target_elevs = [-20, 10, 40]
    distractor_elevs = [-20, -10, 0, 10, 20, 30, 40]
    target_locs = [(azim, elev) for azim in azims for elev in target_elevs]
    distractor_locs = [(azim, elev) for azim in azims for elev in distractor_elevs]

    for t in target_locs:
        for d in distractor_locs:
            for s in ['same', 'opposite']:
                if t[0]*d[0] >= 0:
                    cross = False
                else:
                    cross = True
                a_delta = d[0] - t[0]
                e_delta = d[1] - t[1]
                trial = (t, d, s, cross, a_delta, e_delta)
                trials.append(trial)
    return trials

## Remap names in manifest to gen trial dicts locally 

In [5]:

manifest = pandas.read_pickle('full_eval_trial_manifest_new_fnames.pdpkl')

In [6]:
manifest.keys()

Index(['distractor_client_id', 'distractor_clip_dur_in_s',
       'distractor_clip_end_in_s', 'distractor_clip_start_in_s',
       'distractor_corpus', 'distractor_gender', 'distractor_gender_int',
       'distractor_split', 'distractor_split_int', 'distractor_sr',
       'distractor_src_fn', 'distractor_total_file_duration_in_s',
       'distractor_word', 'src_ix', 'client_id', 'clip_dur_in_s',
       'clip_end_in_s', 'clip_start_in_s', 'corpus', 'gender', 'gender_int',
       'split', 'split_int', 'sr', 'src_fn', 'total_file_duration_in_s',
       'word', 'cue_word', 'cue_src_ix', 'cue_client_id', 'cue_src_fn',
       'cue_clip_start_in_s', 'cue_clip_end_in_s', 'gender_cond_td',
       'cue_clip_dur_in_s'],
      dtype='object')

In [13]:
manifest.cue_src_fn = manifest.cue_src_fn.str.replace('/om/user/imgriff/datasets/spatial_audio_pipeline/assets/',
                                '/Users/mcdermottspeakerarray/Documents/binaural_cocktail_party/sounds/')

In [15]:
manifest.distractor_src_fn = manifest.distractor_src_fn.str.replace('/om/user/imgriff/datasets/spatial_audio_pipeline/assets/',
                                '/Users/mcdermottspeakerarray/Documents/binaural_cocktail_party/sounds/')

In [ ]:
manifest.src_fn = manifest.src_fn.str.replace('/om/user/imgriff/datasets/spatial_audio_pipeline/assets/',
                                '/Users/mcdermottspeakerarray/Documents/binaural_cocktail_party/sounds/')

In [16]:
manifest.to_pickle("full_eval_trial_manifest_new_fnames_local_fnames.pdpkl")

In [25]:
from pathlib import Path
fn_exits = lambda x: Path(x).exists()
print(manifest.src_fn.map(fn_exits).all())
print(manifest.distractor_src_fn.map(fn_exits).all())
print(manifest.cue_src_fn.map(fn_exits).all())



True
True
True


## Get trials locally

In [26]:
manifest = pandas.read_pickle('full_eval_trial_manifest_new_fnames_local_fnames.pdpkl')

In [27]:
def create_experiment(manifest, num_blocks=4):
    """
    create experiment with 4 blocks
    """
    experiment = []
    azim_elev_combos = pickle.load(open('azim_elev_combos.pkl', 'rb'))
    for i in range(num_blocks):
        block = draw_block(azim_elev_combos)
        random.shuffle(block)
        experiment.append(block)

    words = list(manifest.word.unique())
    random.shuffle(words)

    same_gender = 'male_male'
    opp_gender = 'male_female'
    i = 0
    array_manifest = []
    for block in experiment:
        for j, trial in enumerate(block):
            trial_word = words[i]
            i += 1

            if trial[2] == 'same':
                manifest_idx = manifest[(manifest['word'] == trial_word) & (manifest['gender_cond_td'] == f'{same_gender}')].index[0]
                true_cond = same_gender
                if same_gender == 'male_male':
                    same_gender = 'female_female'
                else:
                    same_gender = 'male_male'

            else:
                manifest_idx = manifest[(manifest['word'] == trial_word) & (manifest['gender_cond_td'] == f'{opp_gender}')].index[0]
                true_cond = opp_gender
                if opp_gender == 'male_female':
                    opp_gender = 'female_male'
                else:
                    opp_gender = 'male_female'
            trial_row = manifest.iloc[manifest_idx]
            cue_src_fn = trial_row['cue_src_fn']
            target_src_fn = trial_row['src_fn']
            distractor_src_fn = trial_row['distractor_src_fn']
            distractor_word = trial_row['distractor_word']
            block[j] = block[j] + (trial_word, true_cond, manifest_idx, distractor_word)
            array_manifest.append((trial[0], trial[1], cue_src_fn, target_src_fn, distractor_src_fn))

    experiment_data = dict()
    for i, block in enumerate(experiment):
        experiment_data[f'block_{i}'] = dict()
        block_dict = experiment_data[f'block_{i}']
        for j, trial in enumerate(block):
            block_dict[f'trial_{j}'] = {'target_loc': trial[0],
                                        'distractor_loc': trial[1],
                                        'sex_cond': trial[7],
                                        'crossed': trial[3],
                                        'azim_delta': trial[4],
                                        'elev_delta': trial[5],
                                        'target_word': trial[6],
                                        'distractor_word': trial[9]}

    return experiment_data, array_manifest

In [31]:
def create_new_experiment(manifest, num_azim_blocks=4, num_elev_blocks=2):
    """
    create experiment with two types of blocks
    """
    experiment = []
    for i in range(num_azim_blocks):
        block = draw_block_0_azim()
        random.shuffle(block)
        experiment.append(block)

    for i in range(num_elev_blocks):
        block = draw_block_elev()
        random.shuffle(block)
        experiment.append(block)

    words = list(manifest.word.unique())
    random.shuffle(words)

    same_gender = 'male_male'
    opp_gender = 'male_female'
    i = 0
    array_manifest = []
    for block in experiment:
        for j, trial in enumerate(block):
            try:
                trial_word = words[i]
            except:
                i = 0
                trial_word = words[i]
            i += 1

            if trial[2] == 'same':
                manifest_idx = manifest[(manifest['word'] == trial_word) & (manifest['gender_cond_td'] == f'{same_gender}')].index[0]
                true_cond = same_gender
                if same_gender == 'male_male':
                    same_gender = 'female_female'
                else:
                    same_gender = 'male_male'

            else:
                manifest_idx = manifest[(manifest['word'] == trial_word) & (manifest['gender_cond_td'] == f'{opp_gender}')].index[0]
                true_cond = opp_gender
                if opp_gender == 'male_female':
                    opp_gender = 'female_male'
                else:
                    opp_gender = 'male_female'
            trial_row = manifest.iloc[manifest_idx]
            cue_src_fn = trial_row['cue_src_fn']
            target_src_fn = trial_row['src_fn']
            distractor_src_fn = trial_row['distractor_src_fn']
            distractor_word = trial_row['distractor_word']
            block[j] = block[j] + (trial_word, true_cond, manifest_idx, distractor_word)
            array_manifest.append((trial[0], trial[1], cue_src_fn, target_src_fn, distractor_src_fn))

    experiment_data = dict()
    for i, block in enumerate(experiment):
        experiment_data[f'block_{i}'] = dict()
        block_dict = experiment_data[f'block_{i}']
        for j, trial in enumerate(block):
            block_dict[f'trial_{j}'] = {'target_loc': trial[0],
                                        'distractor_loc': trial[1],
                                        'sex_cond': trial[7],
                                        'crossed': trial[3],
                                        'azim_delta': trial[4],
                                        'elev_delta': trial[5],
                                        'target_word': trial[6],
                                        'distractor_word': trial[9]}
    trial_dict = {i:vals for i,vals in enumerate(array_manifest)}
    return experiment_data, array_manifest, trial_dict

In [33]:
experiment, array_manifest, trial_dict = create_new_experiment(manifest, num_azim_blocks=3, num_elev_blocks=0)

In [34]:
[len(experiment[key]) for key in experiment.keys()]

[50, 50, 50]

In [35]:
# write out manifests 
import pickle 
from pathlib import Path
# get n files in output dir 
out_dir = Path('speaker_array_manifests/pilot_v02')
out_dir.mkdir(exist_ok=True, parents=True)
n_files = len(list(out_dir.glob('*manifest.pkl')))

part_ix = f"participant_{n_files+1:03d}"
print(part_ix)

with open(out_dir / f'{part_ix}_pilot_azimiuth_expt_v02_meta.pkl', 'wb') as f:
    pickle.dump(experiment, f)

with open(out_dir / f'{part_ix}_pilot_azimiuth_expt_v02_array_manifest.pkl', 'wb') as f:
    pickle.dump(array_manifest, f)

with open(out_dir / f'{part_ix}_pilot_azimiuth_expt_v02_trial_dict.pkl', 'wb') as f:
    pickle.dump(trial_dict, f)
    
# get target key list 
out_path = Path("expmt_keys")

word_key = [trial['target_word'] for block in experiment.values() for trial in block.values()]

# save as json 
import json 
with open(out_path / f"{part_ix}_key.json", "w") as f:
    json.dump(word_key, f)



participant_001
